In [1]:
import sys
import import_ipynb
from scipy.sparse.linalg import spsolve
from pathlib import Path

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps  
    from fem_processing import boundary_conditions
    from fem_pos_processing import graph_results, fem_analysis
    print("Modules imports were successful!")
except ModuleNotFoundError as en:
    print(f"Modules were not found: {en}")
except ImportError as en:
    print(f"Error in import: {en}")
    
#run ../setup_project.py

Project root added to sys.path: C:\Users\adilt\OneDrive\05 GIT\PPGEE-2024.2-MEF
Todas as propriedades foram verificadas com sucesso para os elementos P1!
Todas as propriedades foram verificadas com sucesso para os elementos P2!
Todas as propriedades foram verificadas com sucesso para os elementos P3!
Todas as propriedades foram verificadas com sucesso para os elementos Q1!
Todas as propriedades foram verificadas com sucesso para os elementos Q2!
Modules imports were successful!


# Project 1: Poisson Problem in a rectangular domain

Considere um problema de _Poisson_ 2D em $\Omega = [0,1]^2$

$$
-\nabla \cdot \left( \nabla u(x,y) \right) = f(x,y)
\tag{1}
$$

onde,
$$
f(x,y) = 2\pi^2 \, sin(\pi x) \, sin(\pi y)
\tag{2}
$$

cujas condições de contorno são $u=0$ em $\partial \Omega$.

A solução analítica deste problema é

$$
u(x,y) = sin(\pi x) \, sin(\pi y)
\tag{3}
$$

Implemente um programa de elementos finitos para resolver o Problema de Valor de Contorno $(1)$.   

Usar malhas com densidades diferentes e verificar a convergência da sua solução à medida que refinna a malha. Calcule os erros na norma $L_2$ e na norma de energia. A melhor forma de verificar a convergência é traçar gráfios em escala _log-log_ da norma do erro em função de h. Se tudo estiver correto, em escala _log-log_ você obterá retas cujas inclinações fornecerão as taxas de convergência.

# `setTypeProblem()`

In [2]:
def setTypeProblem(FINITE_ELEMENT, h):
    # Create mesh using gmsh
    BOUNDARY = [{'tag': 101, 'type': 'Dirichlet', 'value': 0.0, 'name': 'omega'}]
    MATERIAL = [{'tag': 201, 'name': 'free_space', 'a_constant': 1}]
    
    # Create mesh Structure Data from gmsh
    mesh_data = ps.create_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, h, auto_save=False)
    
    # Get FEM solution
    Agr, fgr, Mgr, Pgr, Qgr = boundary_conditions.reduced_global_matrices(FINITE_ELEMENT, mesh_data)
    uhr = spsolve(Agr.tocsr(), fgr.toarray())
    uh =  boundary_conditions.global_potentials_solution(mesh_data, uhr)
    return uh, mesh_data

# Convergence Analysis

In [3]:
element_types = [('Triangle', 1), ('Triangle', 2), ('Triangle', 3)]
#element_types = [('Quadrangle', 1), ('Quadrangle', 2)]
u = lambda x, y: ps.set_analytical_solution(x, y)
mesh_sizes = [1 / (2**i) for i in range(2, 6)]

# Inicialização dos dicionários para armazenar erros acumulados para cada tipo de elemento
errors_l2 = {et: [] for et in element_types}
errors_energy = {et: [] for et in element_types}

for h in mesh_sizes:  
    for FINITE_ELEMENT in element_types:  
        # Resolve a solução numérica uh para o tipo de elemento e refinamento de malha
        uh, mesh_data = setTypeProblem(FINITE_ELEMENT, h)  
        
        # Calcula o erro L2, o erro de energia e a energia total
        l2, energy, _ = fem_analysis.errors_and_energy(FINITE_ELEMENT, mesh_data, uh, u)

        # Acumula os erros nos dicionários correspondentes
        errors_l2[FINITE_ELEMENT].append(l2)
        errors_energy[FINITE_ELEMENT].append(energy)

# Graphics

In [4]:
graph_results.plot_convergence(mesh_sizes, errors_l2, model='rectangular', name='L2')
graph_results.plot_convergence(mesh_sizes, errors_energy, model='rectangular', name='Energy')

Arquivo salvo em: c:\Users\adilt\OneDrive\05 GIT\PPGEE-2024.2-MEF\p11_poisson_problem\pos_processing\pictures\rectangular_convergence_L2.svg
Arquivo salvo em: c:\Users\adilt\OneDrive\05 GIT\PPGEE-2024.2-MEF\p11_poisson_problem\pos_processing\pictures\rectangular_convergence_Energy.svg


### Figure 1: Error Convergence for $L_2$ norm
<figure>
    <img src="pos_processing/pictures/rectangular_convergence_L2.svg" alt="Fig.1" style="width:80%;" />
    <figcaption>Figure 1: Convergence of the numerical solution for $L_2$ norm of the error</figcaption>
</figure>

### Figure 2: Error Convergence for _Energy_ norm
<figure>
    <img src="pos_processing/pictures/rectangular_convergence_Energy.svg" alt="Fig.2" style="width:80%;" />
    <figcaption>Figure 2: Convergence of the numerical solution for Energy norm of the error</figcaption>
</figure>

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.